# Types

Types define the kind of data a program uses, such as numbers, text, or logical values like `true` or `false`. They describe the shape and behavior of data, helping the program and developer understand how data should be used. All data is stored in memory as a series of bits (1s and 0s), and it's absolutely essential for your app to understand what the position of each those bits means. Being able to write and read bits to represent a particular type of data is called encoding and decoding.

### Why Use Types?
- **Prevent Errors**: Catch mistakes early, such as assigning text to a number or calling a non-existent property.
- **Improve Clarity**: Clearly communicate the purpose of data to others and to future you.
- **Leverage Tools**: Unlock features like autocomplete and inline error checking, making development faster and smoother.

***Note***: Types are not a substitute for runtime checks. For example, TypeScript won't prevent invalid data received from APIs unless you validate it explicitly.

## Primitive Types

Primitive types are the simplest forms of data in TypeScript. They represent single values and are stored in memory differently depending on their type.

Each primitive type is stored in memory differently. Here's a quick overview:

| Type       | Memory Representation                                                                 |
|------------|---------------------------------------------------------------------------------------|
| `boolean`  | 1 bit (`0` or `1`), padded to 8 bits for alignment                                    |
| `null`     | 32-bit or 64-bit pointer (e.g., `0x00000000`)                                         |
| `undefined`| Internal token, often `0xFFFFFFFF`                                                    |
| `number`   | 64 bits (1 sign bit, 11 exponent bits, 52 mantissa bits)                              |
| `bigint`   | Dynamically allocated bits based on the size of the integer                           |
| `string`   | UTF-16 encoding, 16 bits (2 bytes) per character                                      |

In this section, we will explore each primitive type, how it is stored, and provide examples with their binary representations.

## How They Actually Work

### `boolean`
- Represents **true or false** values.
- Stored as a single bit (`0` for `false`, `1` for `true`) but often padded to a full byte (8 bits) for alignment.

#### Examples:
1. `true` → Binary: `00000001`
2. `false` → Binary: `00000000`

### `null`
- Represents the **intentional absence of any value**.
- Typically stored as a 32-bit or 64-bit (depending on the CPU architechture) pointer pointing to a null location in memory (e.g., `0x00000000`).

#### Examples:
- `null` (32-bit) → Binary: `00000000 00000000 00000000 00000000`
- `null` (64-bit) → Binary: `00000000 00000000 00000000 00000000 00000000 00000000 00000000 00000000`

***Note***: Use `null` to represent explicitly empty values. Avoid confusing it with `undefined`.

### `undefined`
- Represents an **uninitialized state**.
- Internally stored as a special token, often represented as a predefined value like `0xFFFFFFFF` (32 bits).

#### Examples:
- `undefined` → Binary: `11111111 11111111 11111111 11111111`

***Note***: `undefined` is distinct from `null`. Avoid using them interchangeably.

### `number`
- Represents **numeric values** (both integers and floating-point numbers).
- Stored as 64-bit floating-point numbers (IEEE 754 format), consisting of:
  - **1 bit** for the sign (positive/negative).
  - **11 bits** for the exponent (scaling factor).
  - **52 bits** for the mantissa (the actual number).

#### Examples:
- `123` → Binary: `0 10000000101 0101000000000000000000000000000000000000000000000000`
- `-42` → Binary: `1 10000000001 0100000000000000000000000000000000000000000000000000`
- `0.1` → Binary: `0 01111111011 1001100110011001100110011001100110011001100110011010`

***Caveat***: Decimal precision issues can occur due to IEEE 754 representation. For example, `0.1 + 0.2 !== 0.3`.

### `bigint`
This is where things start to get much more complicated and you can really start to see why it's important to know about the type of data you're working with and how TypeScript handles it for you behind the scenes.

The `bigint` type allows for working with integers of arbitrary precision, surpassing the safe range of the `number` type in JavaScript and TypeScript. 

A value you use in your code doesn't actual store the numerical data you want it to represent, instead it stores metadata about how your app should behave in order to access that numerical data.
- Metadata includes:
   - A **tag** to indicate the type.
   - A **sign bit** (`0` for positive, `1` for negative).
   - The **number of allocated chunks**, stored as a 32-bit or 64-bit integer (depending on CPU the architecture).
   - A **memory address** pointing to the first chunk of numerical data.
- The actual data is stored elsewhere in memory as chunks of bits.

#### Examples:
- `42n` → Binary: 
   ```plaintext
   Tag:        10101010 (arbitrary, for illustrative purposes)
   Sign:       0 (positive)
   Chunks:     00000000 00000000 00000000 00000001 (1 chunk)
   Address:    00000000 00000000 11111111 00000000 (example memory location)
   ```
   The address would point to this one chunk in memory:
   
   `00101010`

- `987654321987654321n` → Binary: 
   ```plaintext
   Tag:        10101010 (arbitrary, for illustrative purposes)
   Sign:       0 (positive)
   Chunks:     00000000 00000000 00000000 00000110 (6 chunks)
   Address:    00000000 00000000 11111111 00110000 (arbitrary, for illustrative purposes)
   ```
   The address would point to the first of these six chunks:

   `11011011 00100101 10101101 10111111 10011011 11000101`

- `-123456789n` → Binary: 
   ```plaintext
   Tag:        10101010 (arbitrary, for illustrative purposes)
   Sign:       1 (positive)
   Chunks:     00000000 00000000 00000000 00000110 (4 chunks)
   Address:    00000000 00000000 11111111 01001000 (arbitrary, for illustrative purposes)
   ```
   The address would point to the first of these six chunks:

   `00000111 01011011 11001101 00010101`

***Caveats***:
- `bigint` values are not interoperable with `number` types. Explicit conversion is required for operations between them.
- Small `bigint` values may have disproportionately large overhead compared to their binary representation.

### `string`
This type represents textual data or a "string" of characters. Strings are stored in memory using **UTF-16 encoding**.
- Each character in the Basic Multilingual Plane (BMP) is stored as a 16-bit (2-byte) code unit.
- Characters outside the BMP (e.g., emojis, rare scripts) are represented as **surrogate pairs** — two 16-bit code units.

Memory allocation depends on the length of the string and the characters it contains.

- `'`: Single quotes are often used for regular strings without any special functionality.
- `"`: Double quotes serve a similar purpose to single quotes and are largely interchangeable. They are often chosen for consistency, such as when single quotes are already being used within the text itself.
- ``` ` ```: Known as template literals, backticks provide additional functionality. They support string interpolation, which allows you to insert variables directly into the string by wrapping them in `${}`, and also support multiline strings.

#### Examples:
1. **Basic String (`'Hi'`)**:
   - UTF-16 Encoding:
     ```plaintext
     H → 00000000 01001000
     i → 00000000 01101001
     ```

2. **String with Emoji (`"😊"`)**:
   - UTF-16 Encoding (surrogate pair):
     ```plaintext
     😊 → U+1F60A (split into two 16-bit units)
          High Surrogate: 11011000 00111111
          Low Surrogate: 11011100 00101010
     ```
3. **Multilingual Example (``` `你好` ```)**:
   - UTF-16 Encoding:
     ```plaintext
     你 → U+4F60 → 01001111 01100000
     好 → U+597D → 01011001 01111101
     ```

***Caveats***:
- Storing large strings can consume significant memory, especially for strings with surrogate pairs. Be mindful of this when handling large text files or APIs returning large string payloads.
- If a string includes non-UTF-16 data (e.g., ASCII bytes), you might encounter issues when processing it. Ensure proper encoding during input/output operations.